In [1]:
from QuTFyP import *
from awg_iq_simulator import *
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib qt5
from matplotlib import interactive
interactive(True)

In [2]:
# Unit tests 
import unittest
import numpy as np

class WidgetTestCase(unittest.TestCase):
    def setUp(self):
        self.qed = TransmonQED()
        self.qed.d = [2, 4]
        self.qed.initialize_operators()
    def tearDown(self):
        del self.qed
    def testMultipliers(self):
        #n = self.qed.mutipliers_sqr
        n_check = np.asarray(
                  [[[0, 1],
                    [0, 1],
                    [0, 1],
                    [0, 1]],
                   [[0, 0],
                    [1, 0],
                    [2, 0],
                    [3, 0]]]);
        #sqrt_n_check = np.sqrt(n_check)
        n_str = np.array2string(self.qed.multipliers_sqr)
        n_check_str = np.array2string(n_check)
        #sqrt_n_str = np.array2string(self.qed.multipliers)
        #sqrt_n_check_str = np.array2string(sqrt_n_check)
        assert n_check == np.asarray(), 'qed.multipliers_sqr failed check against predefined.\nqed.multipliers_sqr:'+n_str+'\nCheck value:'+n_check_str

In [3]:
unittest.main()

E
ERROR: C:\Users\ilia\AppData\Roaming\jupyter\runtime\kernel-da049ff1-feb2-44bd-8048-ddcde7eed699 (unittest.loader._FailedTest)
----------------------------------------------------------------------
AttributeError: module '__main__' has no attribute 'C:\Users\ilia\AppData\Roaming\jupyter\runtime\kernel-da049ff1-feb2-44bd-8048-ddcde7eed699'

----------------------------------------------------------------------
Ran 1 test in 0.006s

FAILED (errors=1)


SystemExit: True

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
sess = tf.Session()
gi = [[1,0,i] for i in range(ntraj)]  # A list of coordinates to update.
gv = [1.0]*ntraj  # A list of values corresponding to the respective
                # coordinate in indices.
psi = tf.Variable(tf.cast(tf.sparse_tensor_to_dense(tf.SparseTensor(gi, gv, d+[ntraj])), cdtype))
#tf.assign(psi, )
#g = tf.zeros(d, dtype=tf.complex)
#g[*tuple([0]*len(d))] = 1
simulation_time = 5 # nanoseconds
#simulation_time=100
dt = 0.0025

# Control signals
cutoff = 0.375
clock = 1
nop = 6
amplitude = 1

p = pulses(channels={'ex': awg_iq_emulator(clock, cutoff, nop, amplitude),
                     'ro': awg_iq_emulator(clock, cutoff, nop, amplitude)})
#drive_amplitudes = np.linspace(0, 50, ntraj)
drive_amplitudes = 1.
drive_frequencies = np.linspace(5.1, 6.1, ntraj)
#drive_frequencies = np.linspace(9.4, 9.6, ntraj)

controls = {'ex_x':{'carrier':drive_frequencies, 'phi':0,        'coupling':[{}, {'y':-drive_amplitudes}], 'envelope':p.channels['ex'].get_x},
            'ro_x':{'carrier':9.5, 'phi':0,        'coupling':[{}, {'y':-1}], 'envelope':p.channels['ro'].get_x},
            'ex_y':{'carrier':drive_frequencies, 'phi':np.pi/2., 'coupling':[{}, {'x': drive_amplitudes}], 'envelope':p.channels['ex'].get_y},
            'ro_y':{'carrier':9.5, 'phi':np.pi/2., 'coupling':[{}, {'x': 1}], 'envelope':p.channels['ro'].get_y}}

#seq = [p.ps('ex', 32, p.gauss_hd, 1, 0, 8, -0.1),
#       p.ps(channel=None, length=50),
#       p.ps('ex', 32, p.gauss_hd, 1, 0, 8, -0.1),
#       p.ps('ro', 100, p.rect, 1e-2),
#       p.ps(channel=None, length=50)]
#seq = [#p.ps('ro', 5, p.rect, 1e-1),
#       p.ps(channel=None, length=10)]
seq = [{'ex':p.rect('ex', 5, 10.), 'ro':p.rect('ex', 5, 1e-4)}]
p.set_seq(seq)
p.channels['ex'].resample_waveform(dt/2.)
p.channels['ro'].resample_waveform(dt/2.)

tsteps = numpy.arange(0, simulation_time, dt, dtype=numpy.float32)
expectations = numpy.zeros((len(d)*3+1, len(tsteps), ntraj), dtype=numpy.float32)
quantum_noise = numpy.zeros((len(tsteps), ntraj, len(d), 3), dtype=numpy.float32)
#jumps = numpy.zeros((len(tsteps), ntraj), dtype=numpy.float)
masks = []
#jump_probabilities = []
x = tf.placeholder(dtype=psi.dtype, shape=psi.shape)
t = tf.placeholder(dtype=tf.float32, shape=[])
#psi = tf.Tensor(g)
iteration = odeint_iteration(psi, t)
evaluator = expect_sp(psi)


#options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
#run_metadata = tf.RunMetadata()
sess.run(psi.initializer)
t_id = 0
fig = plt.figure('Average photon count in qubit')
line, = plt.plot(drive_frequencies, np.mean(expectations[2,:,:], axis=0))
plt.autoscale(True)
plt.show()
for tit in tqdm(tsteps):
    #psi_new, expectations_new, jump_proba, jumps_new, masks_new = sess.run(iteration, feed_dict={t:tit})
    psi_new, expectations_new, jumps_new, jump_proba, masks_new, x_jumped = sess.run(iteration, feed_dict={t:tit})
    #psi_new, expectations_new, quantum_noise_new = sess.run(iteration, feed_dict={t:tit})
    #print (psi_new, sess.run(ap(am(psi, ax=1),ax=1)))
    expectations[:,t_id,:] = expectations_new
    
    #quantum_noise[t_id,:,:,:] = quantum_noise_new
    
    #jump_probabilities.append(jump_proba[0,:])
    #print (expectations_new[:,0])
    #print (jumps_new[0,:])
    #jumps[t_id,:] = jumps_new
    masks.append(masks_new)
    if t_id%500==0:
        line.set_ydata(np.mean(expectations[2,:t_id,:], axis=0))
        fig.canvas.draw()
        plt.gca().relim()
        plt.pause(0.01)
    t_id+=1
sess.close()

In [ ]:


simulation_time = 5 # nanoseconds
#simulation_time=100
dt = 0.0025

# Control signals
cutoff = 0.375
clock = 1
nop = 6
amplitude = 1

p = pulses(channels={'ex': awg_iq_emulator(clock, cutoff, nop, amplitude),
                     'ro': awg_iq_emulator(clock, cutoff, nop, amplitude)})
#drive_amplitudes = np.linspace(0, 50, ntraj)
drive_amplitudes = 1.
drive_frequencies = np.linspace(5.1, 6.1, ntraj)
#drive_frequencies = np.linspace(9.4, 9.6, ntraj)

controls = {'ex_x':{'carrier':drive_frequencies, 'phi':0,        'coupling':[{}, {'y':-drive_amplitudes}], 'envelope':p.channels['ex'].get_x},
            'ro_x':{'carrier':9.5, 'phi':0,        'coupling':[{}, {'y':-1}], 'envelope':p.channels['ro'].get_x},
            'ex_y':{'carrier':drive_frequencies, 'phi':np.pi/2., 'coupling':[{}, {'x': drive_amplitudes}], 'envelope':p.channels['ex'].get_y},
            'ro_y':{'carrier':9.5, 'phi':np.pi/2., 'coupling':[{}, {'x': 1}], 'envelope':p.channels['ro'].get_y}}

#seq = [p.ps('ex', 32, p.gauss_hd, 1, 0, 8, -0.1),
#       p.ps(channel=None, length=50),
#       p.ps('ex', 32, p.gauss_hd, 1, 0, 8, -0.1),
#       p.ps('ro', 100, p.rect, 1e-2),
#       p.ps(channel=None, length=50)]
#seq = [#p.ps('ro', 5, p.rect, 1e-1),
#       p.ps(channel=None, length=10)]
seq = [{'ex':p.rect('ex', 5, 10.), 'ro':p.rect('ex', 5, 1e-4)}]
p.set_seq(seq)
p.channels['ex'].resample_waveform(dt/2.)
p.channels['ro'].resample_waveform(dt/2.)

tsteps = numpy.arange(0, simulation_time, dt, dtype=numpy.float32)
expectations = numpy.zeros((len(d)*3+1, len(tsteps), ntraj), dtype=numpy.float32)
quantum_noise = numpy.zeros((len(tsteps), ntraj, len(d), 3), dtype=numpy.float32)
#jumps = numpy.zeros((len(tsteps), ntraj), dtype=numpy.float)
masks = []
#jump_probabilities = []
x = tf.placeholder(dtype=psi.dtype, shape=psi.shape)
t = tf.placeholder(dtype=tf.float32, shape=[])
#psi = tf.Tensor(g)
iteration = odeint_iteration(psi, t)
evaluator = expect_sp(psi)


#options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
#run_metadata = tf.RunMetadata()
sess.run(psi.initializer)
t_id = 0
fig = plt.figure('Average photon count in qubit')
line, = plt.plot(drive_frequencies, np.mean(expectations[2,:,:], axis=0))
plt.autoscale(True)
plt.show()
for tit in tqdm(tsteps):
    #psi_new, expectations_new, jump_proba, jumps_new, masks_new = sess.run(iteration, feed_dict={t:tit})
    psi_new, expectations_new, jumps_new, jump_proba, masks_new, x_jumped = sess.run(iteration, feed_dict={t:tit})
    #psi_new, expectations_new, quantum_noise_new = sess.run(iteration, feed_dict={t:tit})
    #print (psi_new, sess.run(ap(am(psi, ax=1),ax=1)))
    expectations[:,t_id,:] = expectations_new
    
    #quantum_noise[t_id,:,:,:] = quantum_noise_new
    
    #jump_probabilities.append(jump_proba[0,:])
    #print (expectations_new[:,0])
    #print (jumps_new[0,:])
    #jumps[t_id,:] = jumps_new
    masks.append(masks_new)
    if t_id%500==0:
        line.set_ydata(np.mean(expectations[2,:t_id,:], axis=0))
        fig.canvas.draw()
        plt.gca().relim()
        plt.pause(0.01)
    t_id+=1
sess.close()